In [1]:
from y0.dsl import A, B, C, D, One, P, Product, Q, R, S, Sum, T, Variable, W, X, Y, Z, Zero

# Level 1 of Pearl's causal hierarchy (conditioning, marginalization)

Probability $1$ event

In [2]:
One()

Probability $0$ event

In [3]:
Zero()

 Probability of event $A$ occurring

In [4]:
P(A)

A variable can be created with the `y0.dsl.Variable` class. Note that it is clever about subscripting the name if it ends with numbers.

In [5]:
A1 = Variable("A1")
P(A1)

Probability of event $A$ not occuring 

In [6]:
P(~A)

Multiplication property of zero

In [7]:
Zero() * P(A, B)

In [8]:
P(A, B) * Zero()

Multiplicative identity

In [9]:
P(A, B) * One()

In [10]:
One() * P(A, B)

Joint probability of $A$ and $B$

In [11]:
P(A, B)

Probability of event $A$ given event $B$

In [12]:
P(A | B)

Probability of event $A$ not occuring given $B$

In [13]:
P(~A | B)

Probability of event $A$ given $B$ does not occur

In [14]:
P(A | ~B)

Probability event $A$ does not occur given $B$ does not occur

In [15]:
P(~A | ~B)

Marginal Probability that event $B$ occurs in a joint distribution over $A$ and $B$.

In [16]:
Sum[A](P(A, B))

Conditional probability of a joint distribution of $A$ and $B$ given $B$

In [17]:
P(A, B) / Sum[B](P(A, B))

Inverse probability of $A$ given $B$

In [18]:
One() / P(A | B)

Ratio of probabilities

In [19]:
(One() / P(A | B)) * P(A)

Product of probabilities

In [20]:
P(A | B) * P(B)

Complex probabilistic expressions 

In [21]:
Sum[B](P(A | B))

In [22]:
Sum[S, T](P(A | B) * Sum[R](P(C | D)))

In [23]:
Sum[W](P((Y @ ~Z @ W) & X) * P(D) * P(Z @ D) * P(W @ ~X))

In [24]:
(P(A) / P(B)) / (P(C) / P(D))

Probabilistic parameter

In [25]:
Q[A](B)

In [26]:
P(Variable(v) for v in ["A", "B", "C"])

In [27]:
Product([P(A), P(B), P(C)])

In [28]:
Sum(P(Variable(v) for v in ["A", "B", "C"]), frozenset(Variable(v) for v in ["A", "B"]))

# Level 2 of Pearl's causal hierarchy (intervention)

Probability of $Y$ given $do(X)$

In [29]:
P[X](Y)

In [30]:
P(Y @ X)

Post-interventional conditional distribution of $Y$ given $Z$

In [31]:
P[X](Y | Z)

# Level 3 of Pearl's causal hierarchy (counterfactual)

Probability of sufficient causation

In [32]:
P(Y @ X | ~X, ~Y)

Probability of necessary causation

In [33]:
P(~Y @ ~X | X, Y)

Probability of necessary and sufficent causation

In [34]:
P(Y @ X, ~Y @ ~X)

Multiple interventions

In [35]:
P(A @ ~W @ X)